In [24]:
import pandas as pd
import requests
from lxml import html
import re
from pydantic import BaseModel
from openai import OpenAI
from datetime import datetime

url_news = "https://www.tagesschau.de/api2u/news?ressort=Faktenfinder"

In [2]:
response = requests.get(url_news)

print(response.status_code)

if response.status_code == 200:
    payload = response.json()
    print("\t", payload.keys())
    print("\t", payload["news"][0].keys())
    print("\t", len(payload["news"]))
    print("")
    df_news = pd.DataFrame(data=payload["news"])

else:
    print("error:", response.status_code)

200
	 dict_keys(['news', 'regional', 'newStoriesCountLink', 'type'])
	 dict_keys(['sophoraId', 'externalId', 'title', 'date', 'teaserImage', 'tags', 'updateCheckUrl', 'tracking', 'topline', 'firstSentence', 'details', 'detailsweb', 'shareURL', 'geotags', 'regionId', 'regionIds', 'ressort', 'breakingNews', 'type'])
	 14



In [70]:
df_news.to_csv("alle-news"+datetime.now().isoformat()[:19] + ".csv")

In [3]:
df_news.shareURL.shape

(14,)

In [20]:
xpath_to_article = "/html/body/div/main/div/div/article"

with open("key.txt", "r") as key_file:
    openai_key = key_file.read()

alle_faktenchecks = []


class Faktencheck(BaseModel):
    person: str
    claim: str
    correct: bool
    explanation: str

class Faktencheckliste(BaseModel):
    faktenchecks: list[Faktencheck]


for shareURL in df_news.shareURL:
    print(shareURL)

    # fetch raw article
    response = requests.get(shareURL)

    # clean up article
    tree = html.fromstring(response.content)
    elements = tree.xpath(xpath_to_article)

    if elements:
        article = elements[0]
        # Remove all <script> tags within the article element
        for script in article.xpath('.//script'):
            script.getparent().remove(script)

        # Extract text and normalize whitespace
        text = article.text_content()
        text = re.sub(r'\s+', ' ', text).strip()
    else:
        print(shareURL)
        print("Element not found.")
        continue


    # summarize by openai in a structured data format
    client = OpenAI(api_key=openai_key)



    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "Du bist eine neutrale Redaktion, die den Faktencheck zusammenfasst."},
            {"role": "user", "content": r"Fass den Artikel so zusammen, dass du für jeden claim die person, ob es correct ist und was die Erklärung ist aus dem Artikel zusammenfasst. Sei kurz und knapp. Bleib bei dem was der Text sagt: \n\n" + text}
        ],
        response_format=Faktencheckliste,
    )
    
    faktencheckliste = completion.choices[0].message.parsed

    alle_faktenchecks.extend(faktencheckliste.model_dump()["faktenchecks"])

https://www.tagesschau.de/faktenfinder/farbebekennen-habeck-100.html
https://www.tagesschau.de/faktenfinder/farbebekennen-weidel-faktencheck-100.html
https://www.tagesschau.de/faktenfinder/wahlarena-aussagen-faktencheck-100.html
https://www.tagesschau.de/faktenfinder/windenergie-falschbehauptungen-100.html
https://www.tagesschau.de/faktenfinder/merz-scholz-faktencheck-100.html
https://www.tagesschau.de/faktenfinder/kontext/russland-desinformation-110.html
https://www.tagesschau.de/faktenfinder/usa-usaid-fakes-100.html
https://www.tagesschau.de/faktenfinder/kontext/rechte-ki-influencer-100.html
https://www.tagesschau.de/faktenfinder/musk-weidel-102.html
https://www.tagesschau.de/faktenfinder/kontext/merz-cdu-migration-kasernen-100.html
https://www.tagesschau.de/faktenfinder/bundestagswahl-2025-fakes-100.html
https://www.tagesschau.de/faktenfinder/ki-fakes-tagesschau-100.html
https://www.tagesschau.de/faktenfinder/kontext/musk-x-bundestagswahl-100.html
https://www.tagesschau.de/faktenfin

In [29]:
df = pd.DataFrame(alle_faktenchecks)
df.to_csv("alle-personen-"datetime.now().isoformat()[:19] + ".csv")

In [38]:
round(df.groupby("person")['correct'].mean() * 100)

person
AfD                                                     100.0
AfD Mayen-Koblenz                                         0.0
Alice Weidel                                              5.0
Anna Hiller                                             100.0
ArrivalAid Stuttgart                                    100.0
Bundesanstalt für Immobilienaufgaben (BImA)             100.0
Carla Reveland                                            0.0
CeMAS (Center für Monitoring, Analyse und Strategie)    100.0
Christoph Maerz                                         100.0
Deborah Schnabel                                        100.0
Donald Trump                                              0.0
Donald Trump Jr. und Elon Musk                            0.0
Elon Musk                                                14.0
Friedrich Merz                                            0.0
Fälschung über Steinmeier                                 0.0
Josef Holnburger                                          0.0
K

In [64]:
relevant_persons =["Alice Weidel", "Donald Trump", "Elon Musk", "Friedrich Merz", "Karl Lauterbach", "Konstantin von Notz", "Olaf Scholz", "Robert Habeck"]
mask_relevant_persons = df.person.isin(relevant_persons)
round(df[mask_relevant_persons].groupby("person")['correct'].mean() * 100)

person
Alice Weidel             5.0
Donald Trump             0.0
Elon Musk               14.0
Friedrich Merz           0.0
Karl Lauterbach        100.0
Konstantin von Notz      0.0
Olaf Scholz             75.0
Robert Habeck           75.0
Name: correct, dtype: float64

In [65]:
df[mask_relevant_persons].groupby("person")["correct"].count()

person
Alice Weidel           19
Donald Trump            2
Elon Musk               7
Friedrich Merz          5
Karl Lauterbach         1
Konstantin von Notz     1
Olaf Scholz             4
Robert Habeck           4
Name: correct, dtype: int64

In [66]:
df[mask_relevant_persons].to_csv("relevante-personen"+datetime.now().isoformat()[:19] + ".csv")

In [67]:
def emojify_correct(correct):
    if correct:
        return "✅"
    return "❌"

In [68]:
mask_merz = df.person == "Friedrich Merz"
for _, row in df[mask_relevant_persons].iterrows():
    print(row.person)
    print(emojify_correct(row.correct), row.claim)
    print("ℹ️", row.explanation, "\n\n")

Robert Habeck
✅ Wehrpflicht nach skandinavischem Vorbild sei machbar und setze auf Freiwilligkeit.
ℹ️ Christian Richter erklärt, eine Auswahlwehrpflicht ist mit dem Grundgesetz vereinbar. Die Freiwilligkeit ist in Skandinavien eingeschränkt, da ein Musterungsbogen verpflichtend ist. 


Robert Habeck
✅ Fossile Energien sind Kostentreiber im Stromsystem.
ℹ️ Bruno Burger bestätigt, dass fossile Energien wegen des Merit-Order-Prinzips die Strompreise in die Höhe treiben. Gaskraftwerke sind teuer und bestimmen oft den Strompreis. 


Robert Habeck
❌ Die Union kündigt alle notwendigen Klimamaßnahmen auf.
ℹ️ Die Union plant, Maßnahmen aufzugeben, aber gleichzeitig den CO2-Preis als Leitinstrument zu nutzen. Ihr Programm wird als inkonsistent beschrieben. 


Alice Weidel
❌ Deutschland hat die höchsten Energiepreise weltweit.
ℹ️ Deutschland hat hohe Energiepreise, aber sie sind nicht die höchsten weltweit. Laut Global Petrol Prices liegt Deutschland je nach Energieart zwischen den Plätzen 3 und 

In [82]:
df_news.date.str.slice(0,10)

0     2025-02-20
1     2025-02-19
2     2025-02-18
3     2025-02-12
4     2025-02-11
5     2025-02-07
6     2025-02-07
7     2025-02-05
8     2025-02-04
9     2025-02-04
10    2025-02-01
11    2025-01-28
12    2025-01-24
13    2025-01-21
Name: date, dtype: object

In [88]:
pd.to_datetime(df_news.date).iloc[0] - pd.to_datetime(df_news.date).iloc[-1]

Timedelta('30 days 01:33:48.386000')

In [94]:
print(df[df.person == "Carla Reveland"].claim.values)
print(df[df.person == "Carla Reveland"].explanation.values)

['Das Video zeigt eine angebliche Vereidigungszeremonie von AfD-Spitzenkandidatin Alice Weidel als neue Bundeskanzlerin.']
["Das Video ist eine Fälschung, erstellt mit Künstlicher Intelligenz. Das identische Bewegtbildmaterial des Stabsmusikkorps ist auf YouTube zu finden und zeigt in Wirklichkeit das Spielen des Regimentsgrußes, nicht 'L'amour toujours'."]
